# Cleaning the Azure Form recognizer results

In [4]:
from utilities import * 
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [5]:
def dict_parser (x):
    try:
        x['value']
        return x['value'] , x['confidence']
    except: 
        return None, None

def expand_dict(df, col_list):
    for col_name in col_list:
        df[f'{col_name}_VALUE'], df[f'{col_name}_CONFIDENCE']  = zip(*df[col_name].apply(dict_parser))
    return df

def get_city_state (x):
    try:
        x = x.replace('\n', ' ')
        zip_ = re.findall(r'\b\d+\b', x)[0]
        state_ = x.replace(zip_, '').strip()
        return state_, zip_
    except: 
        return x.replace('\n', ' '), None
    
def get_zip (x):
    if x is None:
        return None, None
    try:
        x = x.replace('\n', ' ')
        zip_ = re.findall(r'\b\d+\b', x)[0]
        state_ = x.replace(zip_, '').strip()
        return state_, zip_
    except: 
        return x.replace('\n', ' '), None

def duplicate_remover(x1, x2):
    x = [x1, x2]
    x.append(None)
    xx = list(set(x))
    xx.remove(None)
    if len(xx) == 1 :
        return xx[0]
    if len(xx) == 2:
        return xx[1]
    else:
        return None

In [13]:
with open('../dataset/raw/nist/full_azure_form.pkl', 'rb') as fp:
    result_dict = pickle.load(fp)

In [14]:
result_list =[]
for i,j in result_dict.items():
    
    __  = {'image_name': i,}
    j_ = { k.replace("'", '').replace(".", '').\
          replace("E ZIP", 'ZIP').replace("·", '').\
          replace("-", '').replace("_", '').\
          replace("®", '') : v for k, v in j['result_summary'].items() }
    __.update(j_)
    result_list.append(__)

In [15]:
df = pd.DataFrame(result_list)
df.drop(['NAME', 'NAMR', 'NAMA', 
         'NAMN', 'No',
        "print the following shopsataca i"], axis = 1, inplace = True)
print(df.shape)
print(df.columns)
df.head(2)

(2100, 6)
Index(['image_name', 'DATE', 'CITY', 'STATE', 'ZIP', 'STATE\nZIP'], dtype='object')


,image_name,DATE,CITY,STATE,ZIP,STATE\nZIP
0,f0194_30.png,"{'value': '08/08/89', 'confidence': 0.991}","{'value': 'HUNTINGTON BON,', 'confidence': 0.287}","{'value': 'CA 92648', 'confidence': 0.172}",NaN,NaN
1,f1421_27.png,"{'value': '08/28/89', 'confidence': 0.093}",NaN,NaN,NaN,NaN


In [6]:
df.isnull().sum()

image_name       0
DATE           250
CITY           332
STATE          371
ZIP            974
STATE\nZIP    2099
dtype: int64

In [7]:
df = expand_dict(df, ['DATE', 'CITY', 'STATE','ZIP'])
print(df.shape)
df.head(2)

(2100, 14)


,image_name,DATE,CITY,STATE,ZIP,STATE\nZIP,DATE_VALUE,DATE_CONFIDENCE,CITY_VALUE,CITY_CONFIDENCE,STATE_VALUE,STATE_CONFIDENCE,ZIP_VALUE,ZIP_CONFIDENCE
0,f0194_30.png,"{'value': '08/08/89', 'confidence': 0.991}","{'value': 'HUNTINGTON BON,', 'confidence': 0.287}","{'value': 'CA 92648', 'confidence': 0.172}",NaN,NaN,08/08/89,0.991,"HUNTINGTON BON,",0.287,CA\n92648,0.172,None,NaN
1,f1421_27.png,"{'value': '08/28/89', 'confidence': 0.093}",NaN,NaN,NaN,NaN,08/28/89,0.093,None,NaN,None,NaN,None,NaN


In [8]:
df.isnull().sum()

image_name             0
DATE                 250
CITY                 332
STATE                371
ZIP                  974
STATE\nZIP          2099
DATE_VALUE           250
DATE_CONFIDENCE      250
CITY_VALUE           332
CITY_CONFIDENCE      332
STATE_VALUE          371
STATE_CONFIDENCE     371
ZIP_VALUE            974
ZIP_CONFIDENCE       974
dtype: int64

In [9]:
df['CLEAN_STATE_VALUE'], df['CLEAN_ZIP_VALUE'] = zip(*(df.STATE_VALUE.apply(get_zip)))
print(df.shape)
df.head(2)

(2100, 16)


,image_name,DATE,CITY,STATE,ZIP,STATE\nZIP,DATE_VALUE,DATE_CONFIDENCE,CITY_VALUE,CITY_CONFIDENCE,STATE_VALUE,STATE_CONFIDENCE,ZIP_VALUE,ZIP_CONFIDENCE,CLEAN_STATE_VALUE,CLEAN_ZIP_VALUE
0,f0194_30.png,"{'value': '08/08/89', 'confidence': 0.991}","{'value': 'HUNTINGTON BON,', 'confidence': 0.287}","{'value': 'CA 92648', 'confidence': 0.172}",NaN,NaN,08/08/89,0.991,"HUNTINGTON BON,",0.287,CA\n92648,0.172,None,NaN,CA,92648
1,f1421_27.png,"{'value': '08/28/89', 'confidence': 0.093}",NaN,NaN,NaN,NaN,08/28/89,0.093,None,NaN,None,NaN,None,NaN,None,None


In [10]:
df["ZIP_VALUE_FINAL"] = df.apply(lambda x: duplicate_remover(x["ZIP_VALUE"], x["CLEAN_ZIP_VALUE"]), axis=1)
df["STATE_VALUE_FINAL"] = df.apply(lambda x: duplicate_remover(x["STATE_VALUE"], x["CLEAN_STATE_VALUE"]), axis=1)
print(df.shape)
df.head(2)

(2100, 18)


,image_name,DATE,CITY,STATE,ZIP,STATE\nZIP,DATE_VALUE,DATE_CONFIDENCE,CITY_VALUE,CITY_CONFIDENCE,STATE_VALUE,STATE_CONFIDENCE,ZIP_VALUE,ZIP_CONFIDENCE,CLEAN_STATE_VALUE,CLEAN_ZIP_VALUE,ZIP_VALUE_FINAL,STATE_VALUE_FINAL
0,f0194_30.png,"{'value': '08/08/89', 'confidence': 0.991}","{'value': 'HUNTINGTON BON,', 'confidence': 0.287}","{'value': 'CA 92648', 'confidence': 0.172}",NaN,NaN,08/08/89,0.991,"HUNTINGTON BON,",0.287,CA\n92648,0.172,None,NaN,CA,92648,92648,CA
1,f1421_27.png,"{'value': '08/28/89', 'confidence': 0.093}",NaN,NaN,NaN,NaN,08/28/89,0.093,None,NaN,None,NaN,None,NaN,None,None,None,None


In [11]:
df.isnull().sum()

image_name              0
DATE                  250
CITY                  332
STATE                 371
ZIP                   974
STATE\nZIP           2099
DATE_VALUE            250
DATE_CONFIDENCE       250
CITY_VALUE            332
CITY_CONFIDENCE       332
STATE_VALUE           371
STATE_CONFIDENCE      371
ZIP_VALUE             974
ZIP_CONFIDENCE        974
CLEAN_STATE_VALUE     371
CLEAN_ZIP_VALUE      1419
ZIP_VALUE_FINAL       304
STATE_VALUE_FINAL     371
dtype: int64

In [12]:
df_out = df[['image_name', 
    'DATE_VALUE', 'DATE_CONFIDENCE',
    'CITY_VALUE', 'CITY_CONFIDENCE', 
    'CLEAN_STATE_VALUE', 'STATE_CONFIDENCE', 
    'ZIP_VALUE_FINAL', 'ZIP_CONFIDENCE']].copy()
print(df_out.shape)
# df_out = df_out[df_out['ZIP_VALUE_FINAL'].notna()]
print(df_out.shape)
print(df_out.isnull().sum())
df_out.head(2)

(2100, 9)
(2100, 9)
image_name             0
DATE_VALUE           250
DATE_CONFIDENCE      250
CITY_VALUE           332
CITY_CONFIDENCE      332
CLEAN_STATE_VALUE    371
STATE_CONFIDENCE     371
ZIP_VALUE_FINAL      304
ZIP_CONFIDENCE       974
dtype: int64


,image_name,DATE_VALUE,DATE_CONFIDENCE,CITY_VALUE,CITY_CONFIDENCE,CLEAN_STATE_VALUE,STATE_CONFIDENCE,ZIP_VALUE_FINAL,ZIP_CONFIDENCE
0,f0194_30.png,08/08/89,0.991,"HUNTINGTON BON,",0.287,CA,0.172,92648,NaN
1,f1421_27.png,08/28/89,0.093,None,NaN,None,NaN,None,NaN


In [13]:
df_out.rename(columns={"DATE_VALUE": "DATE", 
                       "CITY_VALUE": "CITY", 
                       "CLEAN_STATE_VALUE": "STATE",
                      "ZIP_VALUE_FINAL":"ZIP"}, errors="raise", inplace = True)
print(df_out.shape)
df_out.head(2)

(2100, 9)


,image_name,DATE,DATE_CONFIDENCE,CITY,CITY_CONFIDENCE,STATE,STATE_CONFIDENCE,ZIP,ZIP_CONFIDENCE
0,f0194_30.png,08/08/89,0.991,"HUNTINGTON BON,",0.287,CA,0.172,92648,NaN
1,f1421_27.png,08/28/89,0.093,None,NaN,None,NaN,None,NaN


In [14]:
df_out.to_csv('../dataset/raw/nist/azure_cleaned.csv', index = False)